In [17]:
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
# %jaxtyping.typechecker beartype.beartype

env: JAX_PLATFORM_NAME=cpu
The jaxtyping extension is already loaded. To reload it, use:
  %reload_ext jaxtyping


In [18]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from tqdm.notebook import trange

from jaxtyping import Array, Bool, Float

from chaogatenn.chaogate import NewChaoGate
from chaogatenn.maps import (
    LogisticMap,
    DuffingMap,
    LorenzMap,
    RosslerMap,
    ChenMap,
    RosslerHyperchaosMap,
)
from chaogatenn.utils import grad_norm

In [19]:
# Training data for the AND gate
X = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=bool)  # Input combinations
AND_Y = jnp.array([0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y = jnp.array([0, 1, 1, 1], dtype=bool)  # OR gate output
XOR_Y = jnp.array([0, 1, 1, 0], dtype=bool)  # XOR
NAND_Y = jnp.array([1, 1, 1, 0], dtype=bool)  # NAND
NOR_Y = jnp.array([1, 0, 0, 0], dtype=bool)  # NOR
XNOR_Y = jnp.array([1, 0, 0, 1], dtype=bool)  # XNOR
Y = XNOR_Y

In [20]:
Map = LogisticMap(a=4.0)
# Map = LorenzMap()
# Map = DuffingMap(
#     alpha=1.0, beta=1.0, delta=0.02, gamma=8.0, omega=0.5, dt=0.01, steps=1000
# )
# Map = ChenMap(dt=1e-5, steps=100)
# Map = RosslerHyperchaosMap()

In [21]:
Map(2)  # type: ignore

-8.0

In [22]:
DELTA_X, DELTA_Y, X0, X_THRESHOLD = jax.random.normal(jax.random.PRNGKey(0), (4,))
chao_gate = NewChaoGate(
    DELTA_X=DELTA_X, DELTA_Y=DELTA_Y, X0=X0, X_THRESHOLD=X_THRESHOLD, Map=Map
)

In [23]:
[chao_gate(x) for x in X]

[Array(0.8072584, dtype=float32),
 Array(0.1401511, dtype=float32),
 Array(7.993785e-05, dtype=float32),
 Array(0.15277262, dtype=float32)]

In [24]:
@eqx.filter_value_and_grad()
def compute_loss(
    chao_gate: NewChaoGate, x: Bool[Array, "batch 2"], y: Bool[Array, "batch"]
) -> Float[Array, ""]:  # noqa: F821
    pred = jax.vmap(chao_gate)(x)
    # binary cross entropy
    return -jnp.mean(y * jnp.log(pred + 1e-15) + (1 - y) * jnp.log(1 - pred + 1e-15))

In [25]:
@eqx.filter_jit
def make_step(
    model: NewChaoGate,
    x: Bool[Array, "dim 2"],
    y: Bool[Array, "dim"],  # noqa: F821
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> (Float[Array, "dim"], NewChaoGate, optax.OptState):  # type: ignore  # noqa: F821
    loss, grads = compute_loss(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    # jax.debug.print(f"{grads, updates}")
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state

In [26]:
optim = optax.adabelief(3e-4)
opt_state = optim.init(eqx.filter(chao_gate, eqx.is_inexact_array))

In [27]:
epochs = 10_000

for epoch in trange(epochs):
    loss, chao_gate, opt_state = make_step(chao_gate, X, Y, optim, opt_state)  # type: ignore
    _, grads = compute_loss(chao_gate, X, Y)
    grad_norm_value = grad_norm(grads)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}, Grad Norm: {grad_norm_value}")

    # if loss < 1e-3:
    #     break

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 0, Loss: 0.5609986186027527, Grad Norm: 2.913334846496582
Epoch 10, Loss: 0.5418010950088501, Grad Norm: 2.8317952156066895
Epoch 20, Loss: 0.5157095193862915, Grad Norm: 2.7182648181915283
Epoch 30, Loss: 0.4849158227443695, Grad Norm: 2.5788655281066895
Epoch 40, Loss: 0.4518531858921051, Grad Norm: 2.4215004444122314
Epoch 50, Loss: 0.41851258277893066, Grad Norm: 2.2537543773651123
Epoch 60, Loss: 0.3863244354724884, Grad Norm: 2.082369327545166
Epoch 70, Loss: 0.3562135696411133, Grad Norm: 1.9130005836486816
Epoch 80, Loss: 0.32868966460227966, Grad Norm: 1.7500680685043335
Epoch 90, Loss: 0.3039478659629822, Grad Norm: 1.5966875553131104
Epoch 100, Loss: 0.281963586807251, Grad Norm: 1.4547477960586548
Epoch 110, Loss: 0.26257404685020447, Grad Norm: 1.3250977993011475
Epoch 120, Loss: 0.24554315209388733, Grad Norm: 1.2078001499176025
Epoch 130, Loss: 0.23060619831085205, Grad Norm: 1.1023706197738647
Epoch 140, Loss: 0.2174987494945526, Grad Norm: 1.0080028772354126
Epoc

In [28]:
print("\nTrained ChaoGate Parameters:")
print(
    f"DELTA: {chao_gate.DELTA_X, chao_gate.DELTA_Y}, X0: {chao_gate.X0}, X_THRESHOLD: {chao_gate.X_THRESHOLD}"
)


Trained ChaoGate Parameters:
DELTA: (Array(2.1609957, dtype=float32), Array(-2.069579, dtype=float32)), X0: 0.4518857002258301, X_THRESHOLD: -6.794766426086426


In [29]:
[
    (
        bool(x1.item()),
        bool(x2.item()),
        (
            chao_gate.Map(
                chao_gate.X0 + x1 * chao_gate.DELTA_X + x2 * chao_gate.DELTA_Y
            )
            > chao_gate.X_THRESHOLD
        ).item(),
    )
    for x1, x2 in X
]

[(False, False, True),
 (False, True, False),
 (True, False, False),
 (True, True, True)]

In [30]:
pred_ys = jax.vmap(chao_gate)(X)
num_correct = jnp.sum((pred_ys > 0.5) == Y)
final_accuracy = (num_correct / len(X)).item()
print(f"final_accuracy={final_accuracy}")

final_accuracy=1.0
